## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-18-15-51-54 +0000,nypost,Memo to Mamdani: OD deaths surged after Canada...,https://nypost.com/2025/10/18/us-news/after-st...
1,2025-10-18-15-51-37 +0000,nypost,"Vice President Vance, Witkoff headed to Israel...",https://nypost.com/2025/10/18/world-news/vice-...
2,2025-10-18-15-51-05 +0000,bbc,No Kings protest begins in New York as million...,https://www.bbc.com/news/articles/c93xgyp1zv4o...
3,2025-10-18-15-46-52 +0000,nypost,Oklahoma student says ‘threat’ after Charlie K...,https://nypost.com/2025/10/18/us-news/oklahoma...
4,2025-10-18-15-34-38 +0000,nyt,‘No Kings’ Protests Against Trump: Photos and ...,https://www.nytimes.com/2025/10/18/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,54
28,new,15
171,ukraine,13
20,gaza,11
98,up,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
124,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...,113
145,2025-10-18-00-06-00 +0000,wsj,President Trump has commuted the sentence of f...,https://www.wsj.com/politics/trump-commutes-se...,107
2,2025-10-18-15-51-05 +0000,bbc,No Kings protest begins in New York as million...,https://www.bbc.com/news/articles/c93xgyp1zv4o...,107
180,2025-10-17-22-26-18 +0000,nypost,Trump orders lying ex-Long Island Rep. George ...,https://nypost.com/2025/10/17/us-news/trump-or...,104
218,2025-10-17-20-49-26 +0000,nypost,"Trump urges Russia, Ukraine to end war ‘where ...",https://nypost.com/2025/10/17/us-news/trump-sa...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
124,113,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...
257,71,2025-10-17-18-19-43 +0000,nypost,Jeffrey Epstein-linked Prince Andrew forced to...,https://nypost.com/2025/10/17/world-news/princ...
76,52,2025-10-18-10-43-49 +0000,nyt,Ukraine Braces for New Talks Without the Lever...,https://www.nytimes.com/2025/10/18/world/europ...
1,46,2025-10-18-15-51-37 +0000,nypost,"Vice President Vance, Witkoff headed to Israel...",https://nypost.com/2025/10/18/world-news/vice-...
129,44,2025-10-18-02-00-54 +0000,latimes,Dozens of 'No Kings' protests will be held ac...,https://www.latimes.com/california/story/2025-...
234,42,2025-10-17-19-56-00 +0000,wsj,Bank of Canada Gov. Tiff Macklem said data nex...,https://www.wsj.com/articles/bank-of-canada-su...
290,42,2025-10-17-16-42-19 +0000,nypost,John Bolton spotted still using White House ba...,https://nypost.com/2025/10/17/us-news/john-bol...
150,42,2025-10-17-23-50-52 +0000,nypost,Salesforce CEO Marc Benioff apologizes for say...,https://nypost.com/2025/10/17/us-news/salesfor...
13,34,2025-10-18-15-15-09 +0000,bbc,Afghanistan pulls out of cricket series after ...,https://www.bbc.com/news/articles/c20pnz01x0eo...
135,32,2025-10-18-01-24-15 +0000,nypost,Leather-wearing Democratic state lawmaker Scot...,https://nypost.com/2025/10/17/us-news/leather-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
